In [119]:
import pandas as pd
import pandas as pd
import requests
import json
import os
from bs4 import BeautifulSoup
import re
from queue import Queue
from threading import Thread
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
from difflib import SequenceMatcher

In [73]:
json1_file = open(os.getcwd() + '/Code/Credentials2.json')
json1_str = json1_file.read()
credential = json.loads(json1_str)
Genius_Token = credential['Genius_API']['Access_Token']

In [120]:
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()


In [99]:
def get_song_id(ix,song_title,artist_title,Genius_Token):
    
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + Genius_Token}
    search_url = base_url + '/search'
    data = {'q': str(song_title) + ' ' + str(artist_title)}
    response = requests.get(search_url, data = data, headers=headers)

    if response.ok == True:
        song_info = response.json()
        if len(song_info['response']['hits']) != 0:
            song_id = song_info['response']['hits'][0]['result']['id']

        else:
            song_id = 'Not Found'
        return (ix,song_id)
    else:
        return "Response Error!"
    

In [74]:
def request_song_info(song_id,Genius_Token):
    base_url = 'https://api.genius.com'
    headers = {'Authorization': 'Bearer ' + Genius_Token}
    song_url = base_url + '/songs/' + str(song_id)
    response = requests.get(song_url, headers = headers)

    return response

In [5]:
df = pd.read_csv(r'Clean Data/lyrics.csv')
audio = pd.read_csv(r'Clean Data/MP3_12057_nov20.csv')

/Users/Stella/miniconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [65]:
df['song_id'] = df['song_id'].apply(lambda x: int(x) if x != 'Not Found' else x)

In [68]:
merged = audio.merge(df,on='song_id',how='left')

In [84]:
verify_list = []
for item in merged.iterrows():
    response = request_song_info(int(item[1]['song_id']),Genius_Token)
    if response.ok == True:
        verify_list.append(response.json()['response']['song']['title'])
    else:
        verify_list.append("Not Found")

In [118]:
if len(verify_list) == len(merged):
    merged['verified_title'] = verify_list


In [112]:
merged['if_verified'] = merged.apply(lambda x: 1 if similar(x['TITLE'],x['verified_title']) >= 0.8 else 0 , axis = 1)

In [113]:
merged['if_verified'].value_counts()

0    6486
1    5571
Name: if_verified, dtype: int64

In [115]:
verified_lyrics = merged[merged['if_verified'] == 1]

In [117]:
verified_lyrics.to_csv(r'verified_lyrics.csv',index=False)